<a href="https://colab.research.google.com/github/bryanbayup/petpoint/blob/main/Copy_of_PetpointWithGeneralConver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Jika Anda menggunakan Google Colab, pastikan untuk meng-clone repository terlebih dahulu
!git clone https://github.com/bryanbayup/petpoint

# Pastikan directory kerja Anda berada di tempat yang benar
import os
os.chdir('/content/petpoint')

Cloning into 'petpoint'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 51 (delta 8), reused 28 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 81.82 KiB | 644.00 KiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
import json
import pandas as pd
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import random
import itertools
from collections import Counter

# Pastikan Anda telah menginstall library yang diperlukan
!pip install stanza transformers sentencepiece

import stanza
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 33.4 MB/s eta 0:00:00


In [ ]:
# List untuk menyimpan pasangan percakapan
conversation_pairs = []

# Fungsi untuk mengekstrak pasangan percakapan dari intents
def extract_conversation_pairs(intents):
    pairs = []
    for intent in intents:
        for utterance in intent['utterances']:
            for answer in intent['answers']:
                pairs.append((utterance, answer))
    return pairs

# Load semua JSON files dari corpus/id/
intents = []
for file in glob.glob('corpus/id/*.json'):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        intents.extend(data)

# Load domain-specific data
with open('kucing_anjing/kucing_anjing.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    intents.extend(data)

# Ekstrak pasangan percakapan
conversation_pairs.extend(extract_conversation_pairs(intents))

# Membuat DataFrame
df = pd.DataFrame(conversation_pairs, columns=['input', 'response'])

In [ ]:
# Load normalization dictionary
normalization_dict = {}
with open('normalization/normalization.txt', 'r', encoding='utf-8') as f:
    for line in f:
        # Check if the line contains the delimiter before splitting
        if '\t' in line:
            slang, normal = line.strip().split('\t')
            normalization_dict[slang] = normal
        else:
            print(f"Warning: Skipping line '{line.strip()}', no tab delimiter found.")

# Function to normalize text
def normalize_text(text):
    words = text.split()
    normalized_words = [normalization_dict.get(word, word) for word in words]
    return ' '.join(normalized_words)

# Load stopwords
with open('normalization/stopword.txt', 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

# Function to remove stopwords
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# Initialize the Indonesian pipeline
stanza.download('id')  # Jika sudah pernah di-download, bisa dilewatkan
nlp = stanza.Pipeline('id', processors='tokenize,pos,lemma', use_gpu=True)

def tokenize_and_lemmatize(text):
    doc = nlp(text)
    lemmas = [word.lemma for sentence in doc.sentences for word in sentence.words]
    return ' '.join(lemmas)

# Function for preprocessing
def preprocess_text(text):
    # Normalisasi
    text = normalize_text(text.lower())
    # Hapus stopwords
    text = remove_stopwords(text)
    # Tokenisasi dan Lematisasi
    text = tokenize_and_lemmatize(text)
    return text

# Terapkan preprocessing
df['input'] = df['input'].apply(preprocess_text)
df['response'] = df['response'].apply(preprocess_text)

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: id (Indonesian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/id/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: id (Indonesian):
| Processor | Package      |
----------------------------
| tokenize  | gsd          |
| mwt       | gsd          |
| pos       | gsd_charlm   |
| lemma     | gsd_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be 

In [ ]:
def build_vocab(sentences, max_vocab_size=5000):
    word_counts = Counter(itertools.chain(*[s.split() for s in sentences]))
    most_common = word_counts.most_common(max_vocab_size)
    idx2word = ['<PAD>', '<SOS>', '<EOS>', '<UNK>'] + [word for word, _ in most_common]
    word2idx = {word: idx for idx, word in enumerate(idx2word)}
    return word2idx, idx2word

# Bangun vocabulary untuk input dan output
input_word2idx, input_idx2word = build_vocab(df['input'])
output_word2idx, output_idx2word = build_vocab(df['response'])

def sentence_to_indices(sentence, word2idx):
    indices = [word2idx.get(word, word2idx['<UNK>']) for word in sentence.split()]
    return indices

# Menambahkan indeks ke DataFrame
df['input_indices'] = df['input'].apply(lambda x: sentence_to_indices(x, input_word2idx))
df['response_indices'] = df['response'].apply(lambda x: [output_word2idx['<SOS>']] + sentence_to_indices(x, output_word2idx) + [output_word2idx['<EOS>']])

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
class ChatDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx], dtype=torch.long), torch.tensor(self.outputs[idx], dtype=torch.long)

# Training Dataset dan DataLoader
train_dataset = ChatDataset(train_df['input_indices'].tolist(), train_df['response_indices'].tolist())
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: x)

# Validation Dataset dan DataLoader
val_dataset = ChatDataset(val_df['input_indices'].tolist(), val_df['response_indices'].tolist())
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: x)

In [ ]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, num_layers=1):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        # x: [batch_size, seq_length]
        embedding = self.embedding(x)
        outputs, (hidden, cell) = self.lstm(embedding)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size, num_layers=1):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        # x: [batch_size]
        x = x.unsqueeze(1)  # [batch_size, 1]
        embedding = self.embedding(x)
        outputs, (hidden, cell) = self.lstm(embedding, (hidden, cell))
        predictions = self.fc(outputs.squeeze(1))
        return predictions, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, output_word2idx):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.output_word2idx = output_word2idx

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.size(0)
        target_len = target.size(1)
        target_vocab_size = len(self.output_word2idx)

        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(device)
        hidden, cell = self.encoder(source)

        x = target[:, 0]

        for t in range(1, target_len):
            output, hidden, cell = self.decoder(x, hidden, cell)
            outputs[:, t] = output
            best_guess = output.argmax(1)
            x = target[:, t] if random.random() < teacher_forcing_ratio else best_guess
        return outputs

In [ ]:
import gensim.downloader as api

# Load pre-trained FastText embeddings
fasttext_model = api.load('fasttext-wiki-news-subwords-300')

# Membuat embedding matrix
embedding_dim = 300
vocab_size = len(input_word2idx)
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, idx in input_word2idx.items():
    try:
        embedding_vector = fasttext_model[word]
        embedding_matrix[idx] = embedding_vector
    except KeyError:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim, ))

# Memperbarui layer embedding pada Encoder
encoder_net.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix), freeze=False)

[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
input_size_encoder = len(input_word2idx)
input_size_decoder = len(output_word2idx)
output_size = len(output_word2idx)
embed_size = 256
hidden_size = 512
num_layers = 1
learning_rate = 0.001
num_epochs = 30

encoder_net = Encoder(input_size_encoder, embed_size, hidden_size, num_layers).to(device)
decoder_net = Decoder(output_size, embed_size, hidden_size, num_layers).to(device)

model = Seq2Seq(encoder_net, decoder_net, output_word2idx).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=output_word2idx['<PAD>'])

In [ ]:
for epoch in range(num_epochs):
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(train_dataloader):
        # Ambil batch data
        inputs, targets = zip(*batch)
        inputs = nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=input_word2idx['<PAD>']).to(device)
        targets = nn.utils.rnn.pad_sequence(targets, batch_first=True, padding_value=output_word2idx['<PAD>']).to(device)

        # Forward pass
        outputs = model(inputs, targets)

        # Reshape untuk menghitung loss
        outputs = outputs[:, 1:].reshape(-1, outputs.shape[2])
        targets = targets[:, 1:].reshape(-1)

        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}')

    # Evaluasi pada Validation Set
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_dataloader):
            inputs, targets = zip(*batch)
            inputs = nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=input_word2idx['<PAD>']).to(device)
            targets = nn.utils.rnn.pad_sequence(targets, batch_first=True, padding_value=output_word2idx['<PAD>']).to(device)

            outputs = model(inputs, targets, teacher_forcing_ratio=0)  # No teacher forcing during evaluation

            outputs = outputs[:, 1:].reshape(-1, outputs.shape[2])
            targets = targets[:, 1:].reshape(-1)

            loss = criterion(outputs, targets)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f'Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}')

Epoch [1/30]
Epoch 1, Training Loss: 4.9803
Epoch 1, Validation Loss: 4.6864
Epoch [2/30]
Epoch 2, Training Loss: 4.3357
Epoch 2, Validation Loss: 4.4400
Epoch [3/30]
Epoch 3, Training Loss: 3.9284
Epoch 3, Validation Loss: 4.2024
Epoch [4/30]
Epoch 4, Training Loss: 3.5980
Epoch 4, Validation Loss: 4.1366
Epoch [5/30]
Epoch 5, Training Loss: 3.2753
Epoch 5, Validation Loss: 3.9849
Epoch [6/30]
Epoch 6, Training Loss: 2.8974
Epoch 6, Validation Loss: 3.9328
Epoch [7/30]
Epoch 7, Training Loss: 2.6071
Epoch 7, Validation Loss: 3.9728
Epoch [8/30]
Epoch 8, Training Loss: 2.4143
Epoch 8, Validation Loss: 3.9303
Epoch [9/30]
Epoch 9, Training Loss: 1.9993
Epoch 9, Validation Loss: 4.0028
Epoch [10/30]
Epoch 10, Training Loss: 1.6743
Epoch 10, Validation Loss: 3.8787
Epoch [11/30]
Epoch 11, Training Loss: 1.3154
Epoch 11, Validation Loss: 3.6530
Epoch [12/30]
Epoch 12, Training Loss: 1.2862
Epoch 12, Validation Loss: 4.0287
Epoch [13/30]
Epoch 13, Training Loss: 0.9107
Epoch 13, Validation 

In [ ]:
def translate_sentence(model, sentence, input_word2idx, output_idx2word, max_length=50):
    model.eval()
    with torch.no_grad():
        # Preprocessing
        sentence = preprocess_text(sentence)
        # Konversi ke indeks
        inputs = torch.tensor([sentence_to_indices(sentence, input_word2idx)], dtype=torch.long).to(device)

        hidden, cell = model.encoder(inputs)
        x = torch.tensor([output_word2idx['<SOS>']], dtype=torch.long).to(device)
        outputs = []
        for _ in range(max_length):
            output, hidden, cell = model.decoder(x, hidden, cell)
            best_guess = output.argmax(1)
            if best_guess.item() == output_word2idx['<EOS>']:
                break
            outputs.append(best_guess.item())
            x = best_guess
    translated_sentence = ' '.join([output_idx2word[idx] for idx in outputs])
    return translated_sentence

In [ ]:
# Load sentiment scores
sentiment_scores = {}

# Load positive sentiment words
with open('sentiment/tsv/positive.tsv', 'r', encoding='utf-8') as f:
    for line in f:
        word, score = line.strip().split('\t')
        sentiment_scores[word] = int(score)

# Load negative sentiment words
with open('sentiment/tsv/negative.tsv', 'r', encoding='utf-8') as f:
    for line in f:
        word, score = line.strip().split('\t')
        sentiment_scores[word] = int(score)

def analyze_sentiment(text):
    words = text.split()
    score = 0
    for word in words:
        score += sentiment_scores.get(word, 0)
    return score

def adjust_response_by_sentiment(response, sentiment_score):
    if sentiment_score < 0:
        response = "Maaf mendengarnya. " + response
    elif sentiment_score > 0:
        response = "Senang mendengarnya! " + response
    return response

def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.type) for ent in doc.ents]
    return entities

def generate_template_response(entities):
    if entities:
        for entity, ent_type in entities:
            if ent_type == 'PER':
                return f"Halo {entity}, apa kabar?"
            elif ent_type == 'ORG':
                return f"Apa kabar dengan {entity}?"
    return None

In [ ]:
def chatbot_response(user_input):
    # Analisis sentimen
    sentiment_score = analyze_sentiment(user_input)

    # Ekstrak entitas
    entities = extract_entities(user_input)

    # Coba generate response berbasis template
    response = generate_template_response(entities)
    if response is None:
        # Jika tidak ada entitas, gunakan model
        response = translate_sentence(model, user_input, input_word2idx, output_idx2word)

    # Sesuaikan respon berdasarkan sentimen
    response = adjust_response_by_sentiment(response, sentiment_score)

    return response

In [ ]:
# Load Pre-trained Model
tokenizer = MBart50TokenizerFast.from_pretrained('facebook/mbart-large-50-many-to-many-mmt')
model_transformer = MBartForConditionalGeneration.from_pretrained('facebook/mbart-large-50-many-to-many-mmt').to(device)
tokenizer.src_lang = "id_ID"
tokenizer.tgt_lang = "id_ID"

def generate_transformer_response(input_text):
    # Preprocessing
    input_text = preprocess_text(input_text)
    # Tokenisasi
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    # Generate
    output_ids = model_transformer.generate(input_ids, num_beams=5, max_length=50, early_stopping=True)
    # Decode
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

# Modifikasi fungsi chatbot_response untuk menggunakan Transformer
def chatbot_response_transformer(user_input):
    # Analisis sentimen
    sentiment_score = analyze_sentiment(user_input)

    # Ekstrak entitas
    entities = extract_entities(user_input)

    # Coba generate response berbasis template
    response = generate_template_response(entities)
    if response is None:
        # Jika tidak ada entitas, gunakan model Transformer
        response = generate_transformer_response(user_input)

    # Sesuaikan respon berdasarkan sentimen
    response = adjust_response_by_sentiment(response, sentiment_score)

    return response

In [ ]:
# Chat loop
print("Selamat datang di Chatbot! Ketik 'exit' untuk keluar.")
while True:
    user_input = input("Anda: ")
    if user_input.lower() == 'exit':
        print("Chatbot: Sampai jumpa!")
        break
    bot_response = chatbot_response(user_input)
    print(f"Chatbot: {bot_response}")

Selamat datang di Chatbot! Ketik 'exit' untuk keluar.
Anda: halo
Chatbot: Good luck.
Anda: kamu siapa
Chatbot: In fact, it's not as if we're going to be able to get rid of it.
Anda: siapa kamu
Chatbot: In fact, it's not as if we're going to be able to get rid of it.
Anda: halo
Chatbot: Good luck.
Anda: siapa kamu
Chatbot: In fact, it's not as if we're going to be able to get rid of it.
Anda: aku bermimpi
Chatbot: Dreams.
Anda: iya
Chatbot: Yeah.
Anda: sakit 
Chatbot: Maaf mendengarnya. pain.
Anda: apa kamu suka bermain bola?
Chatbot: Senang mendengarnya! Do you like to play football?
